In [1]:
import json
import numpy as np
from pprint import pprint
fileName = '/Users/erichsu/LING406/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'

data = []
count = 0
with open(fileName) as f:
    for line in f:
        if count>-1:
            count += 1
            data.append(json.loads(line))

In [2]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [3]:
import nltk
from nltk.probability import *

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences( review, tokenizer, remove_stopwords=False ):

    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    return sentences

In [4]:
def frequencyList(corpus):
    freq = {}
    for i in range(len(corpus)):
        sentences = review_to_sentences(data[i]['text'], tokenizer, True)
        for s in sentences:
            for w in s:
                if freq.has_key(w):
                    freq[w] = freq[w] + 1
                else:
                    freq[w] = 1
    return freq

In [5]:
import pandas as pd
reviews = pd.DataFrame(data)

In [6]:
goodReviews = reviews[reviews['stars']>3].reset_index()
badReviews = reviews[reviews['stars']<=3].reset_index()

In [7]:
GoodFreq = frequencyList(goodReviews)
BadFreq = frequencyList(badReviews)

/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://eatinginpittsburgh.com/2011/02/11/coffee-pittsburgh-part-2/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://eatinginpittsburgh.com/2010/07/07/ive-got-an-it

In [8]:
import operator
sorted_g = sorted(GoodFreq.items(), key=operator.itemgetter(1))
sorted_b = sorted(BadFreq.items(), key=operator.itemgetter(1))

In [9]:
sorted_g.reverse()
sorted_b.reverse()

In [12]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

def word_feats(words):
    return dict([(word, True) for word in words])

In [13]:
goodReview = []
badReview = []
for i in range(len(data)):
    if data[i]['stars']>3:
        goodReview.append(data[i]['text'])
    else:
        badReview.append(data[i]['text'])

In [14]:
posfeats = [(word_feats(review_to_wordlist(f)), 'pos') for f in goodReview]
negfeats = [(word_feats(review_to_wordlist(f)), 'neg') for f in badReview]

/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://parismusings.blogspot.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.thestar.com/travel/northamerica/article/1155723--six-new-meals-in-montreal-the-food-lover-s-paradise" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, an

In [15]:
negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))

train on 1176947 instances, test on 392317 instances


In [16]:
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features(100)

accuracy: 0.544980207332
Most Informative Features
                   ruder = True              neg : pos    =     60.4 : 1.0
       unprofessionalism = True              neg : pos    =     59.6 : 1.0
             insinuating = True              neg : pos    =     55.3 : 1.0
                 biaggio = True              pos : neg    =     47.5 : 1.0
                blandest = True              neg : pos    =     47.1 : 1.0
           argumentative = True              neg : pos    =     40.8 : 1.0
              arrogantly = True              neg : pos    =     40.7 : 1.0
              unreturned = True              neg : pos    =     39.3 : 1.0
              mishandled = True              neg : pos    =     38.0 : 1.0
            disrespected = True              neg : pos    =     37.8 : 1.0
               unethical = True              neg : pos    =     35.8 : 1.0
         unaccommodating = True              neg : pos    =     34.9 : 1.0
          unprofessional = True              neg 

In [17]:
from nltk.classify.scikitlearn import SklearnClassifier

In [30]:
import numpy as np
from nltk.probability import FreqDist
from nltk.classify import SklearnClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfTransformer()),
                     ('chi2', SelectKBest(chi2, k=3000)),
                     ('nb', MultinomialNB())])
classif = SklearnClassifier(pipeline)

In [31]:
classifier = classif.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)

accuracy: 0.828462697258


In [36]:
len(GoodFreq)

60961

In [26]:
sorted_g

[(u'good', 630410),
 (u'place', 594577),
 (u'food', 582533),
 (u'great', 480209),
 (u'like', 468439),
 (u'one', 418360),
 (u'get', 412902),
 (u'time', 373649),
 (u'service', 364439),
 (u'go', 345009),
 (u'would', 332834),
 (u'really', 321822),
 (u'back', 302601),
 (u'also', 247683),
 (u've', 241653),
 (u'nice', 230821),
 (u'us', 226617),
 (u'm', 223055),
 (u'even', 220849),
 (u'got', 217587),
 (u'well', 211994),
 (u'best', 206953),
 (u'always', 195936),
 (u'love', 194722),
 (u'little', 189810),
 (u'much', 179264),
 (u'people', 176734),
 (u'restaurant', 173969),
 (u'pretty', 173954),
 (u'vegas', 173413),
 (u'didn', 172940),
 (u'ordered', 166826),
 (u'first', 165317),
 (u'went', 164459),
 (u'could', 163638),
 (u'never', 161827),
 (u'room', 156601),
 (u'order', 156180),
 (u'came', 155178),
 (u'staff', 152760),
 (u'chicken', 152342),
 (u'try', 151606),
 (u'night', 150539),
 (u'know', 148943),
 (u'better', 147377),
 (u'bar', 145438),
 (u'menu', 144456),
 (u'friendly', 143391),
 (u're', 1430

In [27]:
sorted_b

[(u'good', 317379),
 (u'food', 296661),
 (u'place', 295586),
 (u'great', 239369),
 (u'like', 235237),
 (u'one', 210150),
 (u'get', 208836),
 (u'time', 188772),
 (u'service', 180765),
 (u'go', 173233),
 (u'would', 167553),
 (u'really', 159529),
 (u'back', 149769),
 (u've', 122484),
 (u'also', 121410),
 (u'us', 114390),
 (u'nice', 114052),
 (u'even', 110867),
 (u'm', 109368),
 (u'got', 108672),
 (u'best', 107057),
 (u'well', 105152),
 (u'always', 99927),
 (u'vegas', 97790),
 (u'love', 94741),
 (u'little', 94235),
 (u'much', 91083),
 (u'people', 90278),
 (u'room', 90259),
 (u'pretty', 88762),
 (u'restaurant', 88221),
 (u'didn', 87370),
 (u'could', 82896),
 (u'never', 82687),
 (u'went', 82530),
 (u'ordered', 81803),
 (u'first', 81588),
 (u'night', 77469),
 (u'came', 77174),
 (u'order', 76043),
 (u'chicken', 75954),
 (u'better', 75228),
 (u'know', 75222),
 (u'staff', 74900),
 (u're', 71977),
 (u'try', 71836),
 (u'come', 71784),
 (u'bar', 71685),
 (u'going', 71515),
 (u'make', 71086),
 (u'wa

In [34]:
poscutoff

784594

In [33]:
len(trainfeats)

1176947

In [32]:
len(testfeats)

392317